# Explore the OpenBreweryDB API 

In [ ]:
https://sandbox-api.brewerydb.com/v2/
Sandbox Key:c13c821fb670e2fb8cf9a31b4d335d96

In [ ]:
brewery_url = "https://sandbox-api.brewerydb.com/v2/"
brewery_data = requests.get(brewery_url).json()
pprint(brewery_data)

In [ ]:
# brewery_url = "https://api.openbrewerydb.org/breweries?per_page=25"
# brewery_data = requests.get(brewery_url).json()
# pprint(brewery_data)

In [ ]:
# brewery_df = pd.DataFrame(brewery_data)
# brewery_df

# Exploring the CSV format 

In [ ]:
# Read in the CSV using pandas and save into dataframe 
brewery_df = pd.read_csv("https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv")
brewery_df

In [ ]:
# Overview of brewery dataframe 
brewery_df.info()

In [ ]:
# Identify and count null values in each column in df 
nan_count = brewery_df.isnull().sum()
nan_count[nan_count > 0].sort_values(ascending = False)

In [ ]:
# Drop the columns that contain only null values (notes and tags)
brewery_df = brewery_df.drop(['notes', 'tags'], axis = 1)

### Dropping unecessary columns 
created_at, updated_at

In [ ]:
brewery_df.info()

In [ ]:
brewery_df = brewery_df.drop(['created_at', 'updated_at'], axis = 1)

In [ ]:
brewery_df["name"].value_counts()

### Standardising the zip code column 


In [ ]:
# Check the min and mac length of values in the postal Code column 
postal_check = brewery_df['postal_code'].apply(len)
print(f"Max zip_code length is {postal_check.max()}.")
print(f"Min zip_code length is {postal_check.min()}.")

In [ ]:
# Check the rows where the postal code length is 3 and drop it 
brewery_df[brewery_df.postal_code.str.len() == postal_check.min()]
brewery_df = brewery_df.loc[~(brewery_df['postal_code'] == "TBD")]
brewery_df.head()

In [ ]:
# Get rid of the rows that have 4 digit postal codes and null lat and long values
brewery_df = brewery_df.loc[~((brewery_df['postal_code'].str.len() == 4) &
                             (brewery_df['longitude'].isnull())),:]

In [ ]:
brewery_df.info()

In [ ]:
brewery_df[brewery_df.postal_code.str.len() == 5]

In [ ]:
# Clean the postal codes and retain only the first 5 characters 
brewery_df['postal_code'] = brewery_df['postal_code'].apply(lambda x: x.split("-")[0].strip())

In [ ]:
brewery_df[brewery_df.postal_code.str.len() == 5]

In [ ]:
# Verify that all postal codes have 5 characters only 
postal_check = brewery_df['postal_code'].apply(len)
print(f"Max zip_code length is {postal_check.max()}.")
print(f"Min zip_code length is {postal_check.min()}.")

# Google Maps API to obtain lat and lon lists 

In [1]:
# Import dependencies 
import pandas as pd 
import numpy as np
import requests 
import json
from pprint import pprint

In [5]:
df = pd.read_csv("../../Data/beer.gz", encoding="ISO-8859-1")

df.drop_duplicates(subset = "brewery_id", keep = "first", inplace = True)


In [7]:
df

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
4,4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
10,10,163,Amstel Brouwerij B. V.,2002-01-13 23:09:52,3.0,2.0,3.0,fodeeoz,Light Lager,2.5,2.5,Amstel Light,3.5,436
392,393,1454,Broad Ripple Brew Pub,2009-12-30 21:36:37,4.0,3.5,4.0,JamesS,English Brown Ale,3.5,3.5,Nut Brown,4.7,55097
402,403,850,Moon River Brewing Company,2006-03-07 17:19:13,3.5,3.5,3.0,Jredner,American Porter,3.5,3.5,The Captain's Porter,6.0,2780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518208,1586294,21727,Gattopardo Cervejaria,2009-12-17 02:50:44,4.0,3.5,4.0,DoubleJ,German Pilsener,4.0,3.5,Lager Bossa,4.8,54807
1518211,1586301,6785,Brauerei Lasser GmbH,2004-08-04 20:46:43,3.0,3.5,4.0,Bourre,Dortmunder / Export Lager,4.0,2.5,Lasser Export,5.2,19125
1518212,1586302,12566,Wissey Valley Brewery,2008-06-01 18:54:54,4.0,4.5,4.5,wl0307,English Porter,4.5,4.0,Cherry Porter,4.7,42998
1518213,1586303,7337,Outback Brewery Pty Ltd,2009-02-23 09:38:08,2.5,2.5,3.0,LittleCreature,English Dark Mild Ale,3.0,2.5,Outback Black Opal,4.4,13576


In [ ]:
brewery = df['brewery_name']
brewery.drop_duplicates(keep = 'first', inplace = True) 
brewery_names = brewery.to_numpy()

In [ ]:
brewery_names

In [ ]:
https://maps.googleapis.com/maps/api/place/autocomplete/json?input=Vict&types=(cities)&language=pt_BR&key=YOUR_API_KEY

In [ ]:
# obtaining the location of each developer's office using google maps
lat = []
lng = []

params={
    "inputtype": "textquery",
    "types": "establishment",
    "key": g_key,
}

for name in brewery_names:
    params["inputtype"] = name
    url = "https://maps.googleapis.com/maps/api/place/autocomplete/json?parameters"
    response = requests.get(url, params = params).json()
    
    try:
        place_id = response["predictions"][0]["place_id"]
        url2 = "https://maps.googleapis.com/maps/api/geocode/json?place_id="+place_id+"&key="+g_key
        response2 = requests.get(url2).json()
        lat.append(response2["results"][0]["geometry"]["location"]["lat"])
        lng.append(response2["results"][0]["geometry"]["location"]["lng"])
        
    except:
        lat.append("missing")
        lng.append("missing")

In [ ]:
lng

In [ ]:
# obtaining the location of each developer's office using google maps
lat = []
lng = []

params={
    "inputtype":"textquery",
    "key":g_key,
}

for i in dev_gamecount.index:
    params["input"]=i
    url="https://maps.googleapis.com/maps/api/place/autocomplete/json?parameters"
    response=requests.get(url, params=params).json()
    try:
        place_id=response["predictions"][0]["place_id"]
        url2="https://maps.googleapis.com/maps/api/geocode/json?place_id="+place_id+"&key="+g_key
        response2=requests.get(url2).json()
        lat.append(response2["results"][0]["geometry"]["location"]["lat"])
        lng.append(response2["results"][0]["geometry"]["location"]["lng"])
    except:
        lat.append("missing")
        lng.append("missing")